# An Example

In [1]:
import atomdnn

# 'float32' is used for reading data and train by default, one can set data_type to 'float64' here
atomdnn.data_type = 'float64'

# force and stress are evaluated by default, 
# if one only need to compute potential energy, then set compute_force to false
atomdnn.compute_force = True

# default value is for converting ev/A^3 to GPa
# note that: the predicted positive stress means tension and negative stress means compression
stress_unit_convert = 160.2176 

import numpy as np
import tensorflow as tf
from atomdnn import data
from atomdnn import network
from atomdnn.data import Data
from atomdnn.data import *
from atomdnn.network import Network

In [2]:
# load tensorflow dataset, element_spec is only required for Tensorflow version lower than 2.6

dataset = tf.data.experimental.load('example_tfdataset',element_spec=data.element_spec)

In [3]:
# split the data to training, validation and testing sets

train_dataset, val_dataset, test_dataset = split_dataset(dataset,0.7,0.2,0.1,shuffle=True)

Traning data: 210 images
Validation data: 60 images
Test data: 30 images


In [5]:
# Build the network

elements = ['C']
act_fun = 'relu' # activation function
nfp = get_fingerprints_num(dataset) # number of fingerprints (or descriptors)
arch = [10,10] # NN layers

model = Network(elements=elements,num_fingerprints=nfp,arch=arch,activation_function=act_fun)

InvalidArgumentError: Cannot batch tensors with different shapes in component 0. First element had shape [24] and element 1 had shape [].

In [ ]:
# Train the model 

loss_weights = {'pe' : 0.01, 'force' : 1, 'stress': 0.1}

model.train(train_dataset, val_dataset, batch_size=50, lr=0.01, epochs=5,scaling='std', loss_weights=loss_weights, compute_all_loss=True,shuffle=True)

In [ ]:
# Evaluate using test dataset

model.evaluate(test_dataset)

In [ ]:
# take two images from test dataset and do prediction

input_dict = get_input_dict(test_dataset.take(2))
model.predict(input_dict)

In [ ]:
# save the trained model

descriptor = {'name': 'acsf', 
              'cutoff': 6.5001,
              'etaG2':[0.01,0.025,0.05,0.075,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.8,1,1.5,2,3,5,10], 
              'etaG4': [0.01], 
              'zeta': [0.08,0.1,0.15,0.2,0.3,0.35,0.5,0.6,0.8,1.,1.5,2.,3.0,4.,5.5,7.0,10.0,25.0,50.0,100.0],
              'lambda': [1.0, -1.0]}

save_dir = '/workspace/data/weigao/machine_learning/graphene.tfdnn'
network.save(model,save_dir,descriptor=descriptor)

In [ ]:
# print signature
network.print_signature(save_dir)

In [ ]:
# Load the saved model for continuous training and prediction

imported_model = network.load(save_dir)

In [ ]:
imported_model.train(train_dataset, val_dataset, batch_size=50, lr=0.01, epochs=5,scaling='std', loss_weights=loss_weights, shuffle=True)

In [ ]:
imported_model.evaluate(test_dataset)

In [ ]:
input_dict = get_input_dict(test_dataset.take(2))
imported_model.predict(input_dict)